In [11]:
from matplotlib import pyplot as plt

import pandas as pd
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D

from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense


In [12]:
from tensorflow.python.platform import gfile
x_test = cv2.imread(r'C:\Users\prana\Downloads\classes - Copy\54cfb32c-be28-425e-b808-15cea182be78.jpg')
#x_test = cv2.imread(r'C:\Users\prana\Downloads\classes\canada_thistle\0fd8e951242b22e03cbf4ab456cfbfcca6865595.jpg')
#x_test = cv2.imread(r'C:\Users\prana\Downloads\classes\garlic mustard\0d435e8c243351277e03a6531f7ee99406f7b841.jpg')
    
with tf.Session() as sess:
    with gfile.FastGFile(r'C:\tmp\output_graph.pb', 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        sess.graph.as_default()
        g_in = tf.import_graph_def(graph_def)
    tensor_output = sess.graph.get_tensor_by_name('import/final_result:0')
    tensor_input = sess.graph.get_tensor_by_name('import/DecodeJpeg:0')
    predictions = sess.run(tensor_output, {tensor_input:x_test})
    print(predictions)

[[0.00237187 0.9463326  0.05129559]]


In [ ]:
from tensorflow.python.platform import gfile
with tf.Session() as sess:
    with gfile.FastGFile(r'C:\tmp\output_graph.pb', 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        sess.graph.as_default()
        g_in = tf.import_graph_def(graph_def)
    for op in tf.get_default_graph().get_operations():
        print(str(op.name))
    for n in tf.get_default_graph().as_graph_def().node:
         print(n.name)

In [ ]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the data
df = pd.read_csv("slope-lownoise.txt", sep='\t', header=None)
labels = np.asarray(df[0]).reshape((len(df[0]),-1))
data = np.asarray(df.iloc[:,1:])

# Check data
print(labels.shape)
print(data.shape)

# Normalize segments
normdata = np.empty_like(data).astype('float64')
for row in range(len(data)):
    normdata[row] = (data[row] - min(data[row])) / (max(data[row]) - min(data[row]))

In [ ]:
class CNN(object):

    def __init__(self, model_filepath):

        # The file path of model
        self.model_filepath = model_filepath
        # Initialize the model
        self.load_graph(model_filepath = self.model_filepath)

    def load_graph(self, model_filepath):
        '''
        Lode trained model.
        '''
        print('Loading model...')
        self.graph = tf.Graph()
        self.sess = tf.InteractiveSession(graph = self.graph)

        with tf.gfile.GFile(model_filepath, 'rb') as f:
            graph_def = tf.GraphDef()
            graph_def.ParseFromString(f.read())

        #print('Check out the input placeholders:')
        #nodes = [n.name + ' => ' +  n.op for n in graph_def.node if n.op in ('Placeholder')]
        #for node in nodes:
            #print(node)
        
        # Define input tensor
        #self.input = tf.placeholder(np.float32, shape = [None, 150, 32, 3], name='input')
        self.input = tf.placeholder(np.float32, shape = [150, 150, 3], name='input')
        self.dropout_rate = tf.placeholder(tf.float32, shape = [], name = 'dropout_rate')

        tf.import_graph_def(graph_def, {'input': self.input, 'dropout_rate': self.dropout_rate})
        #tf.import_graph_def(graph_def)

        print('Model loading complete!')

        '''
        # Get layer names
        layers = [op.name for op in self.graph.get_operations()]
        for layer in layers:
            print(layer)
        '''

        '''
        # Check out the weights of the nodes
        weight_nodes = [n for n in graph_def.node if n.op == 'Const']
        for n in weight_nodes:
            print("Name of the node - %s" % n.name)
            print("Value - " )
            print(tensor_util.MakeNdarray(n.attr['value'].tensor))
        '''

    def test(self, data):

        # Know your output node name
        output_tensor = self.graph.get_tensor_by_name("import/final_result:0")
        output = self.sess.run(output_tensor, feed_dict = {self.input: data})

        return output
    

In [ ]:
def test_from_frozen_graph(model_filepath):

    tf.reset_default_graph()

    # Load CIFAR10 dataset
    x_test = cv2.imread(r'C:\Users\prana\Downloads\classes\purple_loosestrife\0cd0f56dc0c4b2668e1fe029512aedacad689e1c.jpg')
    x_test4 = [x_test]
    model = CNN(model_filepath = model_filepath)

    for op in tf.get_default_graph().get_operations():
        #print(str(op.name))
        
    for n in tf.get_default_graph().as_graph_def().node:
        #print(n.name)
            
    test_prediction_onehot = model.test(data = x_test)
    test_prediction = np.argmax(test_prediction_onehot, axis = 1).reshape((-1,1))
    print(test_prediction)

    
test_from_frozen_graph(r'c:\tmp\output_graph.pb')
